In [2]:
import pandas as pd
import numpy as np

df = pd.read_parquet("2024_8_19T15_18_19.parquet")

In [3]:
def rearrange(issn):
    data = list(map(lambda x: (f"{x[0][1]}", f"{x[1][1]}"),issn))
    return data

def split_issn(issn):
    e_issn = None
    p_issn = None
    for i in issn:
        if "print" in i:
            p_issn = i[1]
        elif "electronic" in i:
            e_issn = i[1]
    return p_issn, e_issn

def fix_issn(row):
    issn = row['issn']
    first_pass_issn = rearrange(issn)
    p_issn, e_issn = split_issn(first_pass_issn)
    return p_issn, e_issn
    
def separate_container_title(data):
    title = data
    if isinstance(data, np.ndarray):
        title = ", ".join(data.tolist())
    return title

In [4]:
df['separated_tokens'] = df.token_vocabulary.apply(lambda x: ", ".join(sorted(x)))
df[['print_issn', 'electronic_issn']] =df.apply(fix_issn, axis = 1, result_type='expand')
df['container_title'] = df.container_title.apply(separate_container_title)

df.drop(columns=['issn', 'total_processed_ref_len'], inplace=True)

In [164]:
df[(df.work_type == 'book-chapter') & (df.separated_tokens == 'AlWorafi')].groupby('token_frac_refs').count().sort_values('token_frac_refs', ascending=False)

,DOI,token_vocabulary,cleaned_references_length,total_reference_length,work_type,author,flag,member,title,container_title,separated_tokens,print_issn,electronic_issn
token_frac_refs,,,,,,,,,,,,,
1.000000,16,16,16,16,16,16,16,16,16,16,16,0,0
0.973684,2,2,2,2,2,2,2,2,2,2,2,0,0
0.964286,19,19,19,19,19,19,19,19,19,19,19,0,0
0.918919,1,1,1,1,1,1,1,1,1,1,1,0,0
0.884615,1,1,1,1,1,1,1,1,1,1,1,1,1
0.878378,30,30,30,30,30,30,30,30,30,30,30,0,0
0.876712,1,1,1,1,1,1,1,1,1,1,1,0,0
0.866667,2,2,2,2,2,2,2,2,2,2,2,0,0
0.829268,1,1,1,1,1,1,1,1,1,1,1,0,0


In [52]:
import pandas as pd
import numpy as np

df = pd.read_parquet("2024_8_19T15_18_19.parquet")

domain_stopwords = ['Preface', 'Introduction', 'Obituary', 'OBITUARY', 'Bibliographie',
       'In Memoriam', 'Editorial', 'In memoriam','Dedication',
       'Social administration digest', 'Bibliography', 'IN MEMORIAM',
       'Obituaries', 'Book review', 'Back Matter',
       'References', 'Environmental digest', 'General Assembly',
       'Autobiography', 'OBITUARIES', 'Foreword', 'Introduction to the special issue',
       'Social Administration Digest', 'News',  'Book Review', 'Tribute', 'Recommended practices', 'Selected bibliography','Selected Bibliography',
       'Selected bibliography', 'Présentation','Bericht über Patente','Documentation',
       'Results on top physics by CMS', 'Curriculum Vitae', 'NOTES', 'Notes and Comments','Notes','Discussion','Prologue', 'Literatur'
       ]
df[(df.work_type == 'journal-article') & (df.duplicated('title', keep=False)) & (df.title != "") & (~df.title.isin(domain_stopwords))].shape

(791, 14)

In [59]:
df[(df.work_type == 'journal-article') & (df.duplicated(subset=['title'], keep=False)) & (df.title != "") & (~df.title.isin(domain_stopwords))].shape

(791, 14)

In [61]:
df_dois = df[(df.work_type == 'journal-article') & (df.duplicated('title', keep=False)) & (df.title != "") & (~df.title.isin(domain_stopwords))].DOI.to_list()
df_dois[:3]

['10.3103/s0361521923060022',
 '10.1007/s10751-013-0886-6',
 '10.1007/s10327-013-0488-9']

In [6]:
df[(df.work_type == 'journal-article') & (df.title != '')].columns

Index(['DOI', 'token_vocabulary', 'token_frac_refs',
       'cleaned_references_length', 'total_reference_length', 'work_type',
       'author', 'flag', 'member', 'title', 'container_title',
       'separated_tokens', 'print_issn', 'electronic_issn'],
      dtype='object')

In [56]:
journal_articles_df = df[(df.work_type == 'journal-article')][['DOI', 'separated_tokens', 'token_frac_refs', 'author', 'flag', 'title', 'container_title','print_issn', 'electronic_issn','member']]

In [57]:
journal_articles_df

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,member
0,10.1108/13552520010321497,Drucker,0.500000,"[hays, steven w., russ‐sellers, rebecca]",No,On the margins of public administration?,Journal of Management History,1355-252X,None,140
1,10.1021/ic0201194,Westerhausen,0.588235,"[blair, stuart, izod, keith, clegg, william]",No,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,0020-1669,1520-510X,316
2,10.1108/02656719410051643,supplied,0.586207,"[wisner, joel d., eakins, stan g.]",No,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,0265-671X,None,140
3,10.1108/13665629810209066,Teare,0.730769,"[teare, richard e.]",Yes,Developing a curriculum for organizational lea...,Journal of Workplace Learning,1366-5626,None,140
4,10.1108/03068299710178874,Weber,0.757576,"[oakley, allen]",No,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,0306-8293,None,140
...,...,...,...,...,...,...,...,...,...,...
74669,10.2324/gomu.30.7_532,Rubber,0.500000,None,No,,NIPPON GOMU KYOKAISHI,0029-022X,1884-0442,1162
74670,10.1007/s13765-013-3182-x,Codex,0.518519,"[im, moo-hyeog]",No,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,1738-2203,2234-344X,297
74672,10.2324/gomu.30.12_934,Rubber,0.536585,None,No,,NIPPON GOMU KYOKAISHI,0029-022X,1884-0442,1162
74674,10.1254/fpj1925.34.3_285,Ebenda,0.566667,"[minami, tatsuo]",No,^|^Uuml;ber die Wirkung der verschiedenen Komb...,Nippon Yakubutsugaku Zasshi (Folia Pharmacolog...,0369-4461,2185-307X,332


In [143]:
#def sep_author(row):
a = journal_articles_df.copy()
a['sep_author'] = a[~a.author.isnull()].author.apply(lambda x: ", ".join(list(x)))
a = a[~a.author.isnull()]
a

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,member,sep_author
0,10.1108/13552520010321497,Drucker,0.500000,"[hays, steven w., russ‐sellers, rebecca]",No,On the margins of public administration?,Journal of Management History,1355-252X,None,140,"hays, steven w., russ‐sellers, rebecca"
1,10.1021/ic0201194,Westerhausen,0.588235,"[blair, stuart, izod, keith, clegg, william]",No,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,0020-1669,1520-510X,316,"blair, stuart, izod, keith, clegg, william"
2,10.1108/02656719410051643,supplied,0.586207,"[wisner, joel d., eakins, stan g.]",No,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,0265-671X,None,140,"wisner, joel d., eakins, stan g."
3,10.1108/13665629810209066,Teare,0.730769,"[teare, richard e.]",Yes,Developing a curriculum for organizational lea...,Journal of Workplace Learning,1366-5626,None,140,"teare, richard e."
4,10.1108/03068299710178874,Weber,0.757576,"[oakley, allen]",No,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,0306-8293,None,140,"oakley, allen"
...,...,...,...,...,...,...,...,...,...,...,...
74661,10.2478/s11533-013-0328-3,Skowroński,0.510638,"[malicki, piotr, peña, josé, skowroński, andrzej]",Yes,On the number of terms in the middle of almost...,Open Mathematics,None,2391-5455,374,"malicki, piotr, peña, josé, skowroński, andrzej"
74662,10.1186/1744-8603-9-54,Gagliardino,0.521739,"[caporale, joaquín e, elgart, jorge f, gagliar...",Yes,Diabetes in Argentina: cost and management of ...,Globalization and Health,None,1744-8603,297,"caporale, joaquín e, elgart, jorge f, gagliard..."
74663,10.1186/1748-7161-8-16,"Grivas, TB",0.615385,"[grivas, theodoros b, hresko, michael timothy,...",Yes,The pendulum swings back to scoliosis screenin...,Scoliosis,None,1748-7161,297,"grivas, theodoros b, hresko, michael timothy, ..."
74670,10.1007/s13765-013-3182-x,Codex,0.518519,"[im, moo-hyeog]",No,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,1738-2203,2234-344X,297,"im, moo-hyeog"


In [144]:
a[a.separated_tokens == 'Roukos'].groupby('sep_author').count().sort_values('DOI', ascending=False).head(3)

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,member
sep_author,,,,,,,,,,
"hottenrott, christof",26,26,26,26,26,26,26,26,26,26
"liakakos, theodore",5,5,5,5,5,5,5,5,5,5
"spiliotis, john, zoras, odysseas",3,3,3,3,3,3,3,3,3,3


In [158]:
a[a.separated_tokens == 'Roukos'].shape

(87, 11)

In [153]:
k = set(['hottenrott'])
#a[set(list(a.sep_author).split(', ')).intersection(k)]

def get_author(row):
    if 'hottenrott' in row['sep_author']:
        print(row.DOI)
        return row.DOI

h_dois = a.apply(get_author, axis = 1)

10.1007/s00464-013-2988-2
10.1007/s00464-013-2852-4
10.1007/s00464-012-2650-4
10.1007/s00464-012-2599-3
10.1007/s00464-012-2570-3
10.1007/s00464-012-2467-1
10.1007/s00464-012-2411-4
10.1007/s00464-012-2393-2
10.1007/s00464-012-2316-2
10.1007/s00464-012-2275-7
10.1007/s00464-011-2134-y
10.1007/s00464-011-1965-x
10.1007/s00464-011-1935-3
10.1007/s00464-011-1893-9
10.1007/s00464-011-1894-8
10.1007/s00464-011-1808-9
10.1007/s00464-011-1621-5
10.1007/s00464-010-1547-3
10.1007/s00464-010-1422-2
10.1007/s00464-010-1271-z
10.1007/s00464-010-1215-7
10.1007/s00464-010-1154-3
10.1007/s00464-010-1021-2
10.1007/s00464-010-1022-1
10.1007/s00464-009-0848-x
10.1007/s00464-009-0749-z
10.1007/s00464-009-0747-1
10.1007/s00464-009-0697-7
10.1007/s00464-009-0643-8
10.1007/s00464-009-0523-2
10.1245/s10434-007-9721-0
10.52082/jssm.2021.557
10.1007/s00464-014-3456-3


In [ ]:
h_dois = [
'10.1007/s00464-013-2988-2',
'10.1007/s00464-013-2852-4',
'10.1007/s00464-012-2650-4',
'10.1007/s00464-012-2599-3',
'10.1007/s00464-012-2570-3',
'10.1007/s00464-012-2467-1',
'10.1007/s00464-012-2411-4',
'10.1007/s00464-012-2393-2',
'10.1007/s00464-012-2316-2',
'10.1007/s00464-012-2275-7',
'10.1007/s00464-011-2134-y',
'10.1007/s00464-011-1965-x',
'10.1007/s00464-011-1935-3',
'10.1007/s00464-011-1893-9',
'10.1007/s00464-011-1894-8',
'10.1007/s00464-011-1808-9',
'10.1007/s00464-011-1621-5',
'10.1007/s00464-010-1547-3',
'10.1007/s00464-010-1422-2',
'10.1007/s00464-010-1271-z',
'10.1007/s00464-010-1215-7',
'10.1007/s00464-010-1154-3',
'10.1007/s00464-010-1021-2',
'10.1007/s00464-010-1022-1',
'10.1007/s00464-009-0848-x',
'10.1007/s00464-009-0749-z',
'10.1007/s00464-009-0747-1',
'10.1007/s00464-009-0697-7',
'10.1007/s00464-009-0643-8',
'10.1007/s00464-009-0523-2',
'10.1245/s10434-007-9721-0',
'10.52082/jssm.2021.557',
'10.1007/s00464-014-3456-3']

In [157]:
a[a.DOI.isin(h_dois)].sort_values('token_frac_refs', ascending=False)

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,member,sep_author
15590,10.1007/s00464-009-0848-x,Roukos,0.969697,"[hottenrott, christof]",No,Surgeon’s volume and number of lymph nodes in ...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
14322,10.1007/s00464-010-1271-z,Roukos,0.969697,"[hottenrott, christof]",No,Cost-effectiveness analyses of laparoscopic ve...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
7873,10.1007/s00464-011-2134-y,Roukos,0.967742,"[hottenrott, christof]",No,Histology classification challenges for the en...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
42822,10.52082/jssm.2021.557,"Medicine, Science, Sports",0.965517,"[gronwald, thomas, rogers, bruce, hottenrott, ...",No,Correlation Properties of Heart Rate Variabili...,Journal of Sports Science and Medicine,None,1303-2968,29163,"gronwald, thomas, rogers, bruce, hottenrott, l..."
14641,10.1007/s00464-010-1154-3,Roukos,0.941176,"[hottenrott, c.]",No,Comparative-effectiveness research and good cl...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, c."
5622,10.1007/s00464-012-2393-2,Roukos,0.941176,"[hottenrott, christof]",No,Expanding laparoscopic gastrectomy for gastric...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
9511,10.1007/s00464-011-1965-x,Roukos,0.937500,"[hottenrott, christof]",No,Laparoscopic gastrectomy and impact on recurre...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
12584,10.1007/s00464-011-1621-5,Roukos,0.931035,"[hottenrott, christof]",No,Single-incision laparoscopic surgery for color...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
13617,10.1007/s00464-010-1422-2,Roukos,0.909091,"[hottenrott, christof]",No,Right colectomy: is it a safe and feasible tot...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"
15895,10.1007/s00464-009-0747-1,Roukos,0.906250,"[hottenrott, christof]",No,Optimizing lymphadenectomy in laparoscopic sur...,Surgical Endoscopy,0930-2794,1432-2218,297,"hottenrott, christof"


In [75]:
journal_articles_df[(journal_articles_df.duplicated(subset=['title'], keep=False)) & (journal_articles_df.title != "") & (~journal_articles_df.title.isin(domain_stopwords))].shape


(623, 10)

In [76]:
import pandas as pd
flagged_ja = pd.read_csv('flagged_ja_articles.csv')
unflagged_ja = pd.read_csv('unflagged_journal_articles.csv')

In [77]:
processed_ja_doi = flagged_ja.DOI.to_list() + unflagged_ja.DOI.to_list()
other_journal_articles = df[(~df.DOI.isin(processed_ja_doi)) & (df.work_type == 'journal-article')].copy()
other_journal_articles

,DOI,token_vocabulary,token_frac_refs,cleaned_references_length,total_reference_length,work_type,author,flag,member,title,container_title,separated_tokens,print_issn,electronic_issn
0,10.1108/13552520010321497,[Drucker],0.500000,28,28,journal-article,"[hays, steven w., russ‐sellers, rebecca]",No,140,On the margins of public administration?,Journal of Management History,Drucker,1355-252X,None
1,10.1021/ic0201194,[Westerhausen],0.588235,34,34,journal-article,"[blair, stuart, izod, keith, clegg, william]",No,316,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,Westerhausen,0020-1669,1520-510X
2,10.1108/02656719410051643,[supplied],0.586207,29,29,journal-article,"[wisner, joel d., eakins, stan g.]",No,140,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,supplied,0265-671X,None
3,10.1108/13665629810209066,[Teare],0.730769,26,26,journal-article,"[teare, richard e.]",Yes,140,Developing a curriculum for organizational lea...,Journal of Workplace Learning,Teare,1366-5626,None
4,10.1108/03068299710178874,[Weber],0.757576,33,33,journal-article,"[oakley, allen]",No,140,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,Weber,0306-8293,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74669,10.2324/gomu.30.7_532,[Rubber],0.500000,26,43,journal-article,None,No,1162,,NIPPON GOMU KYOKAISHI,Rubber,0029-022X,1884-0442
74670,10.1007/s13765-013-3182-x,[Codex],0.518519,27,27,journal-article,"[im, moo-hyeog]",No,297,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,Codex,1738-2203,2234-344X
74672,10.2324/gomu.30.12_934,[Rubber],0.536585,41,51,journal-article,None,No,1162,,NIPPON GOMU KYOKAISHI,Rubber,0029-022X,1884-0442
74674,10.1254/fpj1925.34.3_285,[Ebenda],0.566667,30,32,journal-article,"[minami, tatsuo]",No,332,^|^Uuml;ber die Wirkung der verschiedenen Komb...,Nippon Yakubutsugaku Zasshi (Folia Pharmacolog...,Ebenda,0369-4461,2185-307X


In [11]:
'10.1108/13665629810209066' in processed_ja_doi

False

In [78]:
other_journal_articles = other_journal_articles[other_journal_articles.title != ''][['DOI', 'title', 'container_title','print_issn', 'electronic_issn','member']]

In [79]:
other_journal_articles

,DOI,title,container_title,print_issn,electronic_issn,member
0,10.1108/13552520010321497,On the margins of public administration?,Journal of Management History,1355-252X,None,140
1,10.1021/ic0201194,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,0020-1669,1520-510X,316
2,10.1108/02656719410051643,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,0265-671X,None,140
3,10.1108/13665629810209066,Developing a curriculum for organizational lea...,Journal of Workplace Learning,1366-5626,None,140
4,10.1108/03068299710178874,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,0306-8293,None,140
...,...,...,...,...,...,...
74661,10.2478/s11533-013-0328-3,On the number of terms in the middle of almost...,Open Mathematics,None,2391-5455,374
74662,10.1186/1744-8603-9-54,Diabetes in Argentina: cost and management of ...,Globalization and Health,None,1744-8603,297
74663,10.1186/1748-7161-8-16,The pendulum swings back to scoliosis screenin...,Scoliosis,None,1748-7161,297
74670,10.1007/s13765-013-3182-x,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,1738-2203,2234-344X,297


In [35]:
unflagged_ja.head(2)

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,ref_pge,token_counts_by_electronic_issn,token_counts_by_print_issn,token_counts_by_container_title,member
0,10.1007/s10910-019-01078-1,Simos,0.885870,"['konguetsof, a.']",No,Algorithm for the development of families of n...,Journal of Mathematical Chemistry,0259-9791,1572-8897,1.000000,110.0,110.0,110.0,297
1,10.1007/s10910-017-0762-8,Simos,0.774011,"['konguetsof, a.']",No,A generator of families of two-step numerical ...,Journal of Mathematical Chemistry,0259-9791,1572-8897,0.994382,110.0,110.0,110.0,297


In [80]:
flagged_unflagged_duplicate_title_dois = ['10.1002/cca.4280',
'10.1002/cca.4206',
'10.1002/cca.3279',
'10.1002/cca.2965',
'10.47992/ijcsbe.2581.6942.0002',
'10.47992/ijcsbe.2581.6942.0001',
'10.1016/0022-328x(92)83206-w',
'10.1016/0022-328x(84)80581-1',
'10.1007/s10910-023-01456-w',
'10.1007/s10910-022-01433-9',
'10.1007/s10973-012-2365-8',
'10.1007/s10973-008-9838-9',
'10.1007/s10973-014-4114-7']


In [81]:
df1 = flagged_ja[['DOI', 'title', 'author','container_title', 'print_issn','electronic_issn','member']]
df2 = unflagged_ja[['DOI', 'title', 'author','container_title', 'print_issn','electronic_issn','member']]
analyzed_ja = pd.concat([df1,df2], ignore_index=True)
analyzed_ja

,DOI,title,author,container_title,print_issn,electronic_issn,member
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
1,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
4,10.1007/s10910-018-0986-2,New multiple stages multistep method with best...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...,...
2614,10.51847/bnyzig6kji,The Function of Maintaining Body Balance in St...,"['zavalishina, svetlana yuryevna' 'karpov, vla...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2615,10.51847/f8mofsugnz,Functional Changes in the Human Body in the Mo...,"['zavalishina, svetlana yuryevna' 'bakulina, e...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2616,10.51847/excvgchspy,Influence of Regular Sports on the Quality of ...,"['kryazhev, valery dmitrievich' 'dorontsev, al...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2617,10.51847/erj8ymdkpc,Physiological Reaction of the Body of Students...,"['zavalishina, svetlana yuryevna' 'karpov, vla...",Journal Of Biochemical Technology,0974-2328,NaN,28733


In [82]:
analyzed_ja = analyzed_ja[~analyzed_ja.DOI.isin(flagged_unflagged_duplicate_title_dois)].copy()
analyzed_ja

,DOI,title,author,container_title,print_issn,electronic_issn,member
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
1,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
4,10.1007/s10910-018-0986-2,New multiple stages multistep method with best...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...,...
2614,10.51847/bnyzig6kji,The Function of Maintaining Body Balance in St...,"['zavalishina, svetlana yuryevna' 'karpov, vla...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2615,10.51847/f8mofsugnz,Functional Changes in the Human Body in the Mo...,"['zavalishina, svetlana yuryevna' 'bakulina, e...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2616,10.51847/excvgchspy,Influence of Regular Sports on the Quality of ...,"['kryazhev, valery dmitrievich' 'dorontsev, al...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2617,10.51847/erj8ymdkpc,Physiological Reaction of the Body of Students...,"['zavalishina, svetlana yuryevna' 'karpov, vla...",Journal Of Biochemical Technology,0974-2328,NaN,28733


In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

def get_title_doi_info(row):
    doi = row['DOI']
    title = row['title']
    return {doi: title}

def process_title_doi_info(info):
    domain_stopwords = ['Preface', 'Introduction', 'Obituary', 'OBITUARY', 'Bibliographie',
       'In Memoriam', 'Editorial', 'In memoriam',
       'Social administration digest', 'Bibliography', 'IN MEMORIAM',
       'Obituaries', 'Book review', 'Back Matter',
       'References', 'Environmental digest', 'General Assembly',
       'Autobiography', 'OBITUARIES', 'Foreword', 'Introduction to the special issue',
       'Social Administration Digest', 'News',  'Book Review',
       'Selected bibliography', 'Présentation','Bericht über Patente',
       'Results on top physics by CMS', 'Curriculum Vitae', 'NOTES', 'Notes and Comments'
       '100 Jahre Allergie: Clemens von Pirquet – sein Allergiebegriff und das ihm zugrunde liegende Krankheitsverständnis']
    title_doi_info = info.to_list()
    combine_info = {}
    for i in title_doi_info:
        for k, v in i.items():
            str = re.sub(r"\&.*?;","", v)
            str = re.sub("SUB", "", str)
            str = re.sub("SUB", "", str)
            str = re.sub(r"<.*?>", "", str)
            if str not in domain_stopwords:
                combine_info[k] = str
    return combine_info

def get_cosine_sim(data):
    titles = list(data.values())
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(titles)
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    ci = dict(zip(list(data.keys()), titles))
    #cosine_sim_df = pd.DataFrame(cosine_sim, index=c, columns=c)
    cosine_sim_df = pd.DataFrame(cosine_sim, index=list(ci.keys()), columns=list(ci.keys()))
    
    # Display the cosine similarity matrix
    #print(cosine_sim_df)
    
    # Find the most similar pairs (excluding self-similarity)
    np.fill_diagonal(cosine_sim, 0)
    return cosine_sim_df

def get_sentences(row, ci, df):
    doi1 = row['row_name']
    doi2 = row['col_name']
    str1 = ci[doi1]
    str2 = ci[doi2]
    doi1_authors = df[df.DOI == doi1].author.to_list()[0]
    doi2_authors = df[df.DOI == doi2].author.to_list()[0]
    return str1, str2, doi1_authors, doi2_authors
    
def get_most_similar_sentences(df, ci, ref_df,threshold = 0.8):
    # Extract values and row, column names
    arr = df.values
    index_names = df.index
    col_names = df.columns
    
    #  Get indices where such threshold is crossed; avoid diagonal elems
    R,C = np.where(np.triu(arr,1)>=threshold)
    
    # Arrange those in columns and put out as a dataframe
    out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
    df_out = pd.DataFrame(out_arr,columns=['row_name','col_name','cosine_sim'])

    df_out[['doi1_title','doi2_title','doi1_authors','doi2_authors']] = df_out.apply(lambda x: get_sentences(x,ci,ref_df), axis = 1, result_type='expand')
    return df_out

In [84]:
# using HF transformer to get embeddings and determining the cosine similarity from this
# using semantic similarity
# 
from sentence_transformers import SentenceTransformer, util
import numpy as np
def get_semantic_similarity(data):
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    titles = list(data.values())
    ci = dict(zip(list(data.keys()), titles))
    embeddings = model.encode(titles)
    similarities = model.similarity(embeddings, embeddings)
    cosine_sim = np.array(similarities)
    model_sim_df = pd.DataFrame(cosine_sim, index=list(ci.keys()), columns=list(ci.keys()))
    return model_sim_df

/Users/eshadatta/sneaked_references_explorations/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [85]:
title_doi_info = analyzed_ja.apply(get_title_doi_info, axis = 1)
combine_info = process_title_doi_info(title_doi_info)
sem_cosine_sim_df = get_semantic_similarity(combine_info)

/Users/eshadatta/sneaked_references_explorations/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [86]:
sem_df_out = get_most_similar_sentences(sem_cosine_sim_df, combine_info, analyzed_ja, 0.80)

In [87]:
sem_df_out['cosine_sim'] = sem_df_out.cosine_sim.apply(lambda x: round(x, 3))

In [88]:
sem_df_out

,row_name,col_name,cosine_sim,doi1_title,doi2_title,doi1_authors,doi2_authors
0,10.1007/s10910-010-9773-4,10.1007/s10910-010-9728-9,0.942,A family of eight-step methods with vanished p...,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']","['alolyan, ibraheem' 'simos, t. e.']"
1,10.1007/s10910-010-9773-4,10.1007/s10910-010-9718-y,0.903,A family of eight-step methods with vanished p...,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']","['alolyan, ibraheem' 'simos, t. e.']"
2,10.1007/s10910-010-9773-4,10.1007/s10910-011-9862-z,0.953,A family of eight-step methods with vanished p...,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']","['alolyan, ibraheem' 'simos, t. e.']"
3,10.1007/s10910-010-9773-4,10.1007/s10910-016-0714-8,0.918,A family of eight-step methods with vanished p...,High order computationally economical six-step...,"['alolyan, ibraheem' 'simos, t. e.']","['berg, dmitriy b.' 'simos, t. e.']"
4,10.1007/s10910-010-9773-4,10.1007/s10910-016-0711-y,0.902,A family of eight-step methods with vanished p...,Two stages six-step method with eliminated pha...,"['alolyan, ibraheem' 'simos, t. e.']","['medvedev, maxim a.' 'simos, t. e.']"
...,...,...,...,...,...,...,...
3593,10.1016/j.ijleo.2020.165644,10.1016/j.ijleo.2018.06.145,0.805,The luminescence characteristics of multicolor...,The effect of synthetic parameters on the stru...,"['al-bluwi, sarah a.' 'shirbeeny, w.']","['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz..."
3594,10.1016/j.ijleo.2018.06.145,10.1016/j.ijleo.2019.162984,0.847,The effect of synthetic parameters on the stru...,"The structure, optical absorption and luminesc...","['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...","['alshahrie, ahmed' 'al-ghamdi, a.a.' 'bronste..."
3595,10.1016/j.ijleo.2018.06.145,10.1016/j.ijleo.2020.164719,0.815,The effect of synthetic parameters on the stru...,The luminescence behavior of Zn1-xTaxO nanopho...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...","['elsayed, w.' 'alshahrie, ahmed' 'al-ghamdi, ..."
3596,10.1016/j.ijleo.2020.164520,10.1016/j.ijleo.2020.164719,0.866,The luminescence spectroscopy and the mechanis...,The luminescence behavior of Zn1-xTaxO nanopho...,"['shokr, f.']","['elsayed, w.' 'alshahrie, ahmed' 'al-ghamdi, ..."


In [89]:
sem_df_out.rename(columns={'row_name': 'DOI', 'col_name': 'DOI2'}, inplace=True)

In [97]:
sem_df_out.to_csv('flagged_unflagged_semantic_similar_titles.csv', index=False)

In [95]:
def print_titles(id):
    str1 = sem_df_out.loc[id].doi1_title
    str2 = sem_df_out.loc[id].doi2_title
    print('str1: ', str1)
    print('str2: ', str2)

print_titles(3573)

str1:  أدب إعراض النبي محمد صلى الله عليه وسلم دراسة قرآنيّة موضوعيّة
str2:  الكلام المحكي على ألسنة الخلق في القرآن الكريم وعلاقته بكلام الله


In [ ]:
    
title_doi_info = analyzed_ja.apply(get_title_doi_info, axis = 1)
combine_info = process_title_doi_info(title_doi_info)
cosine_sim_df = get_cosine_sim(combine_info)
df_out = get_most_similar_sentences(cosine_sim_df, combine_info)

In [245]:

other_journal_title_doi_info = other_journal_articles.apply(get_title_doi_info, axis = 1)
other_journal_combine_info = process_title_doi_info(other_journal_title_doi_info)
other_journal_cosine_sim_df = get_cosine_sim(other_journal_combine_info)
other_journal_df_out = get_most_similar_sentences(other_journal_cosine_sim_df, other_journal_combine_info)

In [ ]:
analyzed_ja

In [254]:
def print_title(id):
    print(other_journal_df_out.loc[id].str1, " ",other_journal_df_out.loc[id].str2) 

print_title(1)

Influence of Catalytic Poisons on the Selectivity of a Cobalt Catalyst for the Fischer–Tropsch Synthesis in Terms of the Theory of Astoichiometric Components of Catalytic Reactions   Influence of Catalytic Poisons on the Selectivity of a Cobalt Catalyst for the Fischer–Tropsch Synthesis in Terms of the Theory of Astoichiometric Components of Catalytic Reactions


In [250]:
other_journal_df_out = other_journal_df_out[(other_journal_df_out.cosine_sim >= 0.94) & (other_journal_df_out.cosine_sim <=1.0)]
other_journal_df_out.shape

(275, 5)

In [246]:
other_journal_df_out.groupby('row_name').count().sort_values('col_name', ascending=False)
#other_journal_df_out['row_name_counts'] = other_journal_df_out.groupby('row_name')['col_name'].transform('count')
#other_journal_df_out

,col_name,cosine_sim,str1,str2
row_name,,,,
10.4467/23921943rp.23.021.18369,7,7,7,7
10.2494/photopolymer.26.3,6,6,6,6
10.2903/j.efsa.2023.8111,6,6,6,6
10.4467/23921943rp.23.008.18267,6,6,6,6
10.2478/acro-2020-0005,5,5,5,5
...,...,...,...,...
10.1016/s0002-9149(03)00703-3,1,1,1,1
10.1016/s0014-5793(00)01810-x,1,1,1,1
10.1016/s0016-0032(25)91042-3,1,1,1,1


In [253]:
other_journal_df_out.sort_values('cosine_sim', ascending=False)

,row_name,col_name,cosine_sim,str1,str2
1,10.3103/s0361521923060022,10.31857/s0023117723060038,1.0,Influence of Catalytic Poisons on the Selectiv...,Influence of Catalytic Poisons on the Selectiv...
540,10.1002/cca.4156,10.1002/cca.2413,1.0,How does pyronaridine plus artesunate compare ...,How does pyronaridine plus artesunate compare ...
468,10.1038/343405a0,10.1038/341013a0,1.0,The peripatetic fossils: part 5,The peripatetic fossils: part 3
470,10.1038/343305a0,10.1038/341013a0,1.0,The peripatetic fossils: part 4,The peripatetic fossils: part 3
476,10.1016/s1353-2561(03)00078-1,10.1016/s1353-2561(03)00057-4,1.0,New Editorial Board Member,New Editorial Board Member
...,...,...,...,...,...
464,10.1016/j.amc.2003.12.049,10.1016/j.cnsns.2004.01.001,0.943004,"Compactons, solitons and periodic solutions fo...",Nonlinear variants of KdV and KP equations wit...
156,10.3103/s0884591323050045,10.15407/kfnt2023.05.024,0.941986,Physical Effects from the Kyiv Meteoroid: Part 1,Physical effects from the Kyiv meteoroid. 1
696,10.1088/1742-6596/1667/1/012045,10.1016/j.nuclphysbps.2018.03.024,0.941287,XYZ states in BESIII,XYZ states at BESIII
132,10.1016/j.mcm.2011.07.017,10.1007/s10598-010-9057-7,0.941064,Some iterative methods for general nonconvex v...,Some iterative methods for nonconvex variation...


In [ ]:
other_journal_df_out[other_journal_df_out.cosine_sim == 1.0].groupby('str1').count().sort_values('str2', ascending=False).head(50)

In [ ]:
import stopwordsiso as stopwords
english_stopwords = stopwords.stopwords("en")
a = other_journal_df_out[other_journal_df_out.cosine_sim == 1.0].groupby('str1').count().sort_values('col_name', ascending=False).head(50)
domain_stopwords = ['Preface', 'Introduction', 'Obituary', 'OBITUARY', 'Bibliographie',
       'In Memoriam', 'Editorial', 'In memoriam',
       'Social administration digest', 'Bibliography', 'IN MEMORIAM',
       'Obituaries', 'Book review', 'Back Matter',
       'References', 'Environmental digest', 'General Assembly',
       'Autobiography', 'OBITUARIES', 'Foreword',
       'Social Administration Digest', 'News',  'Book Review',
       'Selected bibliography', 
       'Results on top physics by CMS', 
       '100 Jahre Allergie: Clemens von Pirquet – sein Allergiebegriff und das ihm zugrunde liegende Krankheitsverständnis']
a.index

In [118]:
df_out['row_name_counts'] = df_out.groupby('row_name')['col_name'].transform('count')
df_out

,row_name,col_name,cosine_sim,str1,str2,row_name_counts
0,10.1007/s10910-016-0674-z,10.1007/s10910-016-0654-3,0.89315,A new two stages tenth algebraic order symmetr...,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,10.1007/s10910-016-0627-6,0.849606,A new two stages tenth algebraic order symmetr...,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,10.1007/s10910-016-0627-6,0.830857,New two stages high order symmetric six-step m...,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,10.1007/s10910-013-0301-1,0.880075,A hybrid type four-step method with vanished p...,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,10.1007/s10910-013-0227-7,0.867309,A hybrid type four-step method with vanished p...,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...
168,10.1002/cca.4206,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2


In [69]:

title_doi_info = analyzed_ja.apply(get_title_doi_info, axis = 1)
combine_info = process_title_doi_info(title_doi_info)

In [123]:
df_out.rename(columns={'row_name': 'DOI'}, inplace=True)
df_out

,DOI,col_name,cosine_sim,str1,str2,row_name_counts
0,10.1007/s10910-016-0674-z,10.1007/s10910-016-0654-3,0.89315,A new two stages tenth algebraic order symmetr...,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,10.1007/s10910-016-0627-6,0.849606,A new two stages tenth algebraic order symmetr...,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,10.1007/s10910-016-0627-6,0.830857,New two stages high order symmetric six-step m...,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,10.1007/s10910-013-0301-1,0.880075,A hybrid type four-step method with vanished p...,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,10.1007/s10910-013-0227-7,0.867309,A hybrid type four-step method with vanished p...,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...
168,10.1002/cca.4206,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2


In [256]:
other_journal_df_out.rename(columns={'row_name': 'DOI'}, inplace=True)
other_journal_df_out

,DOI,col_name,cosine_sim,str1,str2
1,10.3103/s0361521923060022,10.31857/s0023117723060038,1.0,Influence of Catalytic Poisons on the Selectiv...,Influence of Catalytic Poisons on the Selectiv...
5,10.1007/s10327-013-0483-1,10.3186/jjphytopath.79.131,1.0,Studies on diseases of tropical crops,Studies on diseases of tropical crops
8,10.1051/mmnp/20138505,10.1134/s1560354709010092,0.945301,On Global Bifurcations of Three-dimensional Di...,On global bifurcations in three-dimensional di...
9,10.1016/j.jtbi.2013.09.008,10.1177/1088767912460393,0.986945,Introduction to special issue,Introduction to the Special Issue
11,10.1007/s10511-013-9295-4,10.1007/s10511-013-9277-6,0.960046,Evolution of the magnetic fields of main-seque...,Evolution of the magnetic fields of main-seque...
...,...,...,...,...,...
906,10.1134/s2079978015020028,10.1134/s207997801501001x,1.0,Methods for the determination of glucose in bl...,Methods for the determination of glucose in bl...
907,10.1515/pbj-2015-0007,10.2478/pbj-2014-0023,1.0,"A new species of Microchilus (Goodyerinae, Orc...","A new species of Microchilus (Orchidaceae, Goo..."
909,10.1134/s1064226915020084,10.1134/s1064226914090046,0.964015,Atomic and WA-systems and R-functions applied ...,"Atomic, WA-systems, and R-functions applied in..."
912,10.4011/shikizai1937.80.174,10.4011/shikizai1937.80.267,1.0,Functionalization of Nanoparticles by Grafting...,Functionalization of Nanoparticles by Grafting...


In [90]:
analyzed_ja

,DOI,title,author,container_title,print_issn,electronic_issn,member
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
1,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
4,10.1007/s10910-018-0986-2,New multiple stages multistep method with best...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...,...
2614,10.51847/bnyzig6kji,The Function of Maintaining Body Balance in St...,"['zavalishina, svetlana yuryevna' 'karpov, vla...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2615,10.51847/f8mofsugnz,Functional Changes in the Human Body in the Mo...,"['zavalishina, svetlana yuryevna' 'bakulina, e...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2616,10.51847/excvgchspy,Influence of Regular Sports on the Quality of ...,"['kryazhev, valery dmitrievich' 'dorontsev, al...",Journal Of Biochemical Technology,0974-2328,NaN,28733
2617,10.51847/erj8ymdkpc,Physiological Reaction of the Body of Students...,"['zavalishina, svetlana yuryevna' 'karpov, vla...",Journal Of Biochemical Technology,0974-2328,NaN,28733


In [91]:
sem_combined_df = pd.merge(analyzed_ja, sem_df_out, on='DOI')
sem_combined_df

,DOI,title,author,container_title,print_issn,electronic_issn,member,DOI2,cosine_sim,doi1_title,doi2_title,doi1_authors,doi2_authors
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9728-9,0.942,A family of eight-step methods with vanished p...,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']","['alolyan, ibraheem' 'simos, t. e.']"
1,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9718-y,0.903,A family of eight-step methods with vanished p...,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']","['alolyan, ibraheem' 'simos, t. e.']"
2,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-011-9862-z,0.953,A family of eight-step methods with vanished p...,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']","['alolyan, ibraheem' 'simos, t. e.']"
3,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0714-8,0.918,A family of eight-step methods with vanished p...,High order computationally economical six-step...,"['alolyan, ibraheem' 'simos, t. e.']","['berg, dmitriy b.' 'simos, t. e.']"
4,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0711-y,0.902,A family of eight-step methods with vanished p...,Two stages six-step method with eliminated pha...,"['alolyan, ibraheem' 'simos, t. e.']","['medvedev, maxim a.' 'simos, t. e.']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,10.1016/j.ijleo.2020.165644,The luminescence characteristics of multicolor...,"['al-bluwi, sarah a.' 'shirbeeny, w.']",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2018.06.145,0.805,The luminescence characteristics of multicolor...,The effect of synthetic parameters on the stru...,"['al-bluwi, sarah a.' 'shirbeeny, w.']","['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz..."
3594,10.1016/j.ijleo.2018.06.145,The effect of synthetic parameters on the stru...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2019.162984,0.847,The effect of synthetic parameters on the stru...,"The structure, optical absorption and luminesc...","['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...","['alshahrie, ahmed' 'al-ghamdi, a.a.' 'bronste..."
3595,10.1016/j.ijleo.2018.06.145,The effect of synthetic parameters on the stru...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2020.164719,0.815,The effect of synthetic parameters on the stru...,The luminescence behavior of Zn1-xTaxO nanopho...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...","['elsayed, w.' 'alshahrie, ahmed' 'al-ghamdi, ..."
3596,10.1016/j.ijleo.2020.164520,The luminescence spectroscopy and the mechanis...,"['shokr, f.']",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2020.164719,0.866,The luminescence spectroscopy and the mechanis...,The luminescence behavior of Zn1-xTaxO nanopho...,"['shokr, f.']","['elsayed, w.' 'alshahrie, ahmed' 'al-ghamdi, ..."


In [92]:
sem_combined_df.drop(columns=['doi1_title', 'doi1_authors'], inplace=True)
sem_combined_df

,DOI,title,author,container_title,print_issn,electronic_issn,member,DOI2,cosine_sim,doi2_title,doi2_authors
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9728-9,0.942,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']"
1,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9718-y,0.903,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']"
2,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-011-9862-z,0.953,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']"
3,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0714-8,0.918,High order computationally economical six-step...,"['berg, dmitriy b.' 'simos, t. e.']"
4,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0711-y,0.902,Two stages six-step method with eliminated pha...,"['medvedev, maxim a.' 'simos, t. e.']"
...,...,...,...,...,...,...,...,...,...,...,...
3593,10.1016/j.ijleo.2020.165644,The luminescence characteristics of multicolor...,"['al-bluwi, sarah a.' 'shirbeeny, w.']",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2018.06.145,0.805,The effect of synthetic parameters on the stru...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz..."
3594,10.1016/j.ijleo.2018.06.145,The effect of synthetic parameters on the stru...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2019.162984,0.847,"The structure, optical absorption and luminesc...","['alshahrie, ahmed' 'al-ghamdi, a.a.' 'bronste..."
3595,10.1016/j.ijleo.2018.06.145,The effect of synthetic parameters on the stru...,"['al-ghamdi, ahmed a.' 'al-ameer, s.' 'al-marz...",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2020.164719,0.815,The luminescence behavior of Zn1-xTaxO nanopho...,"['elsayed, w.' 'alshahrie, ahmed' 'al-ghamdi, ..."
3596,10.1016/j.ijleo.2020.164520,The luminescence spectroscopy and the mechanis...,"['shokr, f.']",Optik,0030-4026,NaN,78,10.1016/j.ijleo.2020.164719,0.866,The luminescence behavior of Zn1-xTaxO nanopho...,"['elsayed, w.' 'alshahrie, ahmed' 'al-ghamdi, ..."


In [97]:
doi2 = list(set(sem_combined_df.DOI2.to_list()))
doi2_info = analyzed_ja[analyzed_ja.DOI.isin(doi2)][['DOI', 'author', 'container_title', 'print_issn', 'electronic_issn', 'member']]
doi2_info

,DOI,author,container_title,print_issn,electronic_issn,member
1,10.1007/s10910-010-9728-9,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
7,10.1007/s10910-016-0714-8,"['berg, dmitriy b.' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
8,10.1007/s10910-016-0711-y,"['medvedev, maxim a.' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...
2601,10.1016/0003-2670(56)80070-6,"['(max) bobtelsky, mordechai' 'mayer, bracha']",Analytica Chimica Acta,0003-2670,NaN,78
2603,10.17073/1683-4518-2020-1-28-38,"['pivinsky, yu. e.']",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550
2604,10.17073/1683-4518-2019-11-39-48,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,...",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550
2607,10.1016/0022-328x(88)80026-3,"['constantin filippou, alexander' 'otto fische...",Journal of Organometallic Chemistry,0022-328X,NaN,78


In [98]:
doi2_info.rename(columns={'DOI': 'DOI2', 'author': 'doi2_author', 'container_title': 'doi2_container_title', 'print_issn': 'doi2_print_issn', 'electronic_issn': 'doi2_electronic_issn', 'member': 'doi2_member'}, inplace=True)
doi2_info

,DOI2,doi2_author,doi2_container_title,doi2_print_issn,doi2_electronic_issn,doi2_member
1,10.1007/s10910-010-9728-9,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
7,10.1007/s10910-016-0714-8,"['berg, dmitriy b.' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
8,10.1007/s10910-016-0711-y,"['medvedev, maxim a.' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...
2601,10.1016/0003-2670(56)80070-6,"['(max) bobtelsky, mordechai' 'mayer, bracha']",Analytica Chimica Acta,0003-2670,NaN,78
2603,10.17073/1683-4518-2020-1-28-38,"['pivinsky, yu. e.']",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550
2604,10.17073/1683-4518-2019-11-39-48,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,...",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550
2607,10.1016/0022-328x(88)80026-3,"['constantin filippou, alexander' 'otto fische...",Journal of Organometallic Chemistry,0022-328X,NaN,78


In [101]:
sem_all_info = pd.merge(doi2_info, sem_combined_df, on='DOI2')
sem_all_info

,DOI2,doi2_author,doi2_container_title,doi2_print_issn,doi2_electronic_issn,doi2_member,DOI,title,author,container_title,print_issn,electronic_issn,member,cosine_sim,doi2_title,doi2_authors
0,10.1007/s10910-010-9728-9,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,0.942,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']"
1,10.1007/s10910-010-9718-y,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,0.903,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']"
2,10.1007/s10910-010-9718-y,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,0.895,High algebraic order methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']"
3,10.1007/s10910-011-9862-z,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,0.953,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']"
4,10.1007/s10910-011-9862-z,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,"['alolyan, ibraheem' 'simos, t. e.']",Journal of Mathematical Chemistry,0259-9791,1572-8897,297,0.913,A family of ten-step methods with vanished pha...,"['alolyan, ibraheem' 'simos, t. e.']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,10.17073/1683-4518-2019-11-39-48,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,...",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550,10.17073/1683-4518-2021-9-20-34,Cement-free refractory concretes. Part 11. Col...,"['pivinskii, yu. e.' 'dyakin, p. v.']",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550,0.832,Cementless refractory concrete. Part 2. High a...,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,..."
3594,10.17073/1683-4518-2019-11-39-48,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,...",NOVYE OGNEUPORY (NEW REFRACTORIES),1683-4518,NaN,6550,10.1007/s11148-020-00408-0,Cement-Free Refractory Concretes. Part 2. High...,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,...",Refractories and Industrial Ceramics,1083-4877,1573-9139,297,0.937,Cementless refractory concrete. Part 2. High a...,"['pivinsky, yu. e.' 'dyakin, p. v.' 'grishpun,..."
3595,10.1016/0022-328x(88)80026-3,"['constantin filippou, alexander' 'otto fische...",Journal of Organometallic Chemistry,0022-328X,NaN,78,10.1016/s0022-328x(00)85735-6,Übergangsmetall-carben-komplexe,"['fischer, e.o.' 'pardy, r.b.a.' 'schubert, u.']",Journal of Organometallic Chemistry,0022-328X,NaN,78,0.953,Übergangsmetall-Carbin-Komplexe,"['constantin filippou, alexander' 'otto fische..."
3596,10.1016/0022-328x(88)80026-3,"['constantin filippou, alexander' 'otto fische...",Journal of Organometallic Chemistry,0022-328X,NaN,78,10.1016/s0022-328x(00)88934-2,Übergangsmetall—carbin-komplexe,"['fischer, ernst otto' 'schubert, ulrich']",Journal of Organometallic Chemistry,0022-328X,NaN,78,0.968,Übergangsmetall-Carbin-Komplexe,"['constantin filippou, alexander' 'otto fische..."


In [104]:
sem_all_info.to_csv('flagged_unflagged_semantic_similar_titles.csv', index=False)

In [124]:
y = pd.merge(analyzed_ja, df_out, on='DOI')
y

,DOI,title,container_title,print_issn,electronic_issn,member,col_name,cosine_sim,str1,str2,row_name_counts
0,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0654-3,0.89315,A new two stages tenth algebraic order symmetr...,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.849606,A new two stages tenth algebraic order symmetr...,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,New two stages high order symmetric six-step m...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.830857,New two stages high order symmetric six-step m...,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0301-1,0.880075,A hybrid type four-step method with vanished p...,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0227-7,0.867309,A hybrid type four-step method with vanished p...,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...,...,...,...,...,...
168,10.1002/cca.4206,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2


In [259]:
other_y = pd.merge(other_journal_articles, other_journal_df_out, on='DOI')
other_y.drop(columns=['str1'], inplace=True)
other_y.rename(columns={'col_name': 'DOI2', 'str2': 'DOI2_title', 'row_name_counts': 'DOI_counts'}, inplace=True)
other_y

,DOI,title,container_title,print_issn,electronic_issn,member,DOI2,cosine_sim,DOI2_title
0,10.3103/s0361521923060022,Influence of Catalytic Poisons on the Selectiv...,Solid Fuel Chemistry,0361-5219,1934-8029,1627,10.31857/s0023117723060038,1.0,Influence of Catalytic Poisons on the Selectiv...
1,10.1007/s10327-013-0483-1,Studies on diseases of tropical crops,Journal of General Plant Pathology,1345-2630,1610-739X,297,10.3186/jjphytopath.79.131,1.0,Studies on diseases of tropical crops
2,10.1051/mmnp/20138505,On Global Bifurcations of Three-dimensional Di...,Mathematical Modelling of Natural Phenomena,0973-5348,1760-6101,250,10.1134/s1560354709010092,0.945301,On global bifurcations in three-dimensional di...
3,10.1016/j.jtbi.2013.09.008,Introduction to special issue,Journal of Theoretical Biology,0022-5193,None,78,10.1177/1088767912460393,0.986945,Introduction to the Special Issue
4,10.1007/s10511-013-9295-4,Evolution of the magnetic fields of main-seque...,Astrophysics,0571-7256,1573-8191,297,10.1007/s10511-013-9277-6,0.960046,Evolution of the magnetic fields of main-seque...
...,...,...,...,...,...,...,...,...,...
270,10.1134/s2079978015020028,Methods for the determination of glucose in bl...,Review Journal of Chemistry,2079-9780,2079-9799,137,10.1134/s207997801501001x,1.0,Methods for the determination of glucose in bl...
271,10.1515/pbj-2015-0007,"A new species of Microchilus (Goodyerinae, Orc...",Polish Botanical Journal,None,2084-4352,21052,10.2478/pbj-2014-0023,1.0,"A new species of Microchilus (Orchidaceae, Goo..."
272,10.1134/s1064226915020084,Atomic and WA-systems and R-functions applied ...,Journal of Communications Technology and Elect...,1064-2269,1555-6557,137,10.1134/s1064226914090046,0.964015,"Atomic, WA-systems, and R-functions applied in..."
273,10.4011/shikizai1937.80.174,Functionalization of Nanoparticles by Grafting...,Journal of the Japan Society of Colour Material,0010-180X,None,2426,10.4011/shikizai1937.80.267,1.0,Functionalization of Nanoparticles by Grafting...


In [127]:
#y.rename(columns={'col_name': 'DOI2', 'str2': 'DOI2_title', 'row_name_counts': 'DOI_counts'}, inplace=True)
y.drop(columns=['str1'], inplace=True)
y

,DOI,title,container_title,print_issn,electronic_issn,member,DOI2,cosine_sim,DOI2_title,DOI_counts
0,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0654-3,0.89315,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.849606,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,New two stages high order symmetric six-step m...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.830857,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0301-1,0.880075,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0227-7,0.867309,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...,...,...,...,...
168,10.1002/cca.4206,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,2


In [131]:
y['container_title_counts'] = y.groupby('container_title')['DOI'].transform('count')

In [260]:
other_y['container_title_counts'] = other_y.groupby('container_title')['DOI'].transform('count')

In [264]:
other_y['DOI_counts'] = other_y.groupby('DOI')['title'].transform('count')

In [133]:
y.to_csv('possible_similar_titles_flagged_unflagged_journal_titles.csv', index=False)

In [265]:
other_y.to_csv('possible_similar_titles_other_journal_titles.csv', index=False)

In [ ]:
import pandas as pd
